In [12]:
%load_ext autoreload
%autoreload
notebook = True
skip_inputs = 'y'
%run -i redistricter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


seed_start           (default=100) 
jobs_per_worker      (default=1) 
workers              (default=1) 


Get crosswalks_TX_2010_tabblock   ... tabblock table exists ... success
Get assignments_TX_2020_tabblock  ... tabblock table exists ... success
Get shapes_TX_2020_tabblock       ... tabblock table exists ... success
Get census_TX_2020_tabblock       ... tabblock table exists ... success
Get elections_TX_2020_tabblock    ... tabblock table exists ... success
Get nodes_TX_2020_cntyvtd_cd      ... cntyvtd table exists ... success
Get graph_TX_2020_cntyvtd         ... gpickle exists ... success
hashseed != 0 so results are NOT reproducible and will NOT be saved to BigQuery
I will run seeds [100]
starting seed 100
finished seed 100 with pop_imbalance=65.5 after 14 steps and 0hrs 0min 9.59sec


In [92]:
district_type = 'cd'

tbl = f'cmat-315920.redistricting_results_cook.TX_2020_cntyvtd_cd_seed'
s = "\nunion all\n"
seeds = [s for s in range (100, 116) if s not in [104, 110]]
summary_stack = s.join([f'select * from {tbl}_{str(seed).rjust(4, "0")}_summary' for seed in seeds])
stats_stack   = s.join([f'select * from {tbl}_{str(seed).rjust(4, "0")}_stats'   for seed in seeds])
plans_stack   = s.join([f'select * from {tbl}_{str(seed).rjust(4, "0")}_plans'   for seed in seeds])

query = f"""
select
    B.seed,
    B.plan,
    C.{district_type},
    B.pop_imbalance,
    B.polsby_popper as polsby_popper_plan,
    C.polsby_popper as polsby_popper_district,
    C.total_pop
from (
    select
        *
    from (
        select
            *,
            row_number() over (partition by A.hash order by plan asc, seed asc) as r
        from (
            {summary_stack}
            ) as A
        )
    where r = 1
    ) as B
join (
    select
        *
    from (
        {stats_stack}
        )
    ) as C
on
    B.seed = C.seed and B.plan = C.plan
"""
df = run_query(query)
# 
df#['hash'].value_counts().sort_values(ascending=False)

,seed,plan,cd,pop_imbalance,polsby_popper_plan,polsby_popper_district,total_pop
0,105,1,01,88.179594,20.215682,19.400764,722363
1,105,2,01,84.423077,20.303012,19.400764,722363
2,105,3,01,80.639179,20.131629,19.400764,722363
3,105,4,01,80.639179,20.116386,19.400764,722363
4,105,5,01,80.639179,20.145664,19.400764,722363
...,...,...,...,...,...,...,...
16563,106,36,38,51.809094,18.371053,21.889269,843877
16564,106,37,38,51.369191,18.172050,21.889269,843877
16565,106,38,38,50.333710,18.533837,21.889269,843877
16566,106,39,38,50.333710,18.580757,21.889269,843877


In [101]:
district_type = 'cd'

nodes = f'cmat-315920.redistricting_data.nodes_TX_2020_cntyvtd_cd'
tbl = f'cmat-315920.redistricting_results_cook.TX_2020_cntyvtd_cd_seed'
s = "\nunion all\n"
seeds = [s for s in range (100, 116) if s not in [104, 110]]
summary_stack = s.join([f'select * from {tbl}_{str(seed).rjust(4, "0")}_summary' for seed in seeds])
stats_stack   = s.join([f'select * from {tbl}_{str(seed).rjust(4, "0")}_stats'   for seed in seeds])
plans_stack   = s.join([f'select * from {tbl}_{str(seed).rjust(4, "0")}_plans'   for seed in seeds])

query = f"""
select
    E.seed,
    E.plan,
    E.{district_type},
    max(E.pop_imbalance) as pop_imbalance,
    max(E.polsby_popper_plan) as polsby_popper_plan,
    max(E.polsby_popper_district) as polsby_popper_district,
    max(E.total_pop) as total_pop,
    sum(F.total_1race)
from (
    select
        B.seed,
        B.plan,
        C.{district_type},
        B.pop_imbalance,
        B.polsby_popper as polsby_popper_plan,
        C.polsby_popper as polsby_popper_district,
        C.total_pop,
        D.geoid
    from (
        select
            *
        from (
            select
                *,
                row_number() over (partition by A.hash order by plan asc, seed asc) as r
            from (
                {summary_stack}
                ) as A
            )
        where r = 1
        ) as B
    inner join (
        {stats_stack}
        ) as C
    on
        B.seed = C.seed and B.plan = C.plan
    inner join (
        select
            *
        from (
            {plans_stack}
            )
        ) as D
    on
        C.seed = D.seed and C.plan = D.plan and C.{district_type} = D.{district_type}
    ) as E
inner join
    {nodes} as F
on
    E.geoid = F.geoid
group by
    seed, plan, {district_type}
"""
df = run_query(query)
# 
df#['hash'].value_counts().sort_values(ascending=False)

,seed,plan,cd,pop_imbalance,polsby_popper_plan,polsby_popper_district,total_pop,f0_
0,111,9,02,55.718287,19.195283,12.457524,575591,488720
1,111,21,02,51.275447,17.592821,12.457524,575591,488720
2,111,22,04,51.275447,17.439941,14.673533,949693,849910
3,111,23,04,51.275447,17.282653,14.673533,949693,849910
4,111,28,07,50.307764,18.059005,27.321004,606907,517689
...,...,...,...,...,...,...,...,...
16563,113,1,22,88.179594,19.907970,14.200205,972269,833709
16564,113,8,22,88.179594,17.787377,14.200205,972269,833709
16565,108,1,22,88.179594,19.915859,14.200205,972269,833709
16566,106,7,22,84.423077,19.240416,14.200205,972269,833709


In [105]:
district_type = 'cd'

G.nodes_tbl = f'cmat-315920.redistricting_data.nodes_TX_2020_cntyvtd_cd'
M.tbl = f'cmat-315920.redistricting_results_cook.TX_2020_cntyvtd_cd_seed'

s = "\nunion all\n"
seeds = [s for s in range (100, 116) if s not in [104, 110]]
summary_stack = s.join([f'select * from {tbl}_{str(seed).rjust(4, "0")}_summary' for seed in seeds])
stats_stack   = s.join([f'select * from {tbl}_{str(seed).rjust(4, "0")}_stats'   for seed in seeds])
plans_stack   = s.join([f'select * from {tbl}_{str(seed).rjust(4, "0")}_plans'   for seed in seeds])

cols = [c for c in get_cols(self.nodes) if c not in Levels + District_types + ['geoid', 'county', 'total_pop', 'polygon', 'perim', 'polsby_popper', 'density', 'point']]

query = f"""
select
    B.seed,
    B.plan,
    C.{district_type},
    max(B.pop_imbalance) as pop_imbalance_plan,
    max(B.polsby_popper) as polsby_popper_plan,
    max(C.polsby_popper) as polsby_popper_district,
    max(C.total_pop) as total_pop,
    max(C.total_pop) / sum(E.aland) as density,
    {join_str(1).join([f'sum(E.{c}) as {c}' for c in cols])}
from (
    select
        *
    from (
        select
            *,
            row_number() over (partition by A.hash order by plan asc, seed asc) as r
        from (
            {summary_stack}
            ) as A
        )
    where r = 1
    ) as B
inner join (
    {stats_stack}
    ) as C
on
    B.seed = C.seed and B.plan = C.plan
inner join (
    select
        *
    from (
        {plans_stack}
        )
    ) as D
on
    C.seed = D.seed and C.plan = D.plan and C.{district_type} = D.{district_type}
inner join
    {nodes} as E
on
    D.geoid = E.geoid
group by
    seed, plan, {district_type}
"""
df = run_query(query)
# 
df#['hash'].value_counts().sort_values(ascending=False)

,seed,plan,cd,pop_imbalance_plan,polsby_popper_plan,polsby_popper_district,total_pop,density,density2,f0_
0,114,22,29,54.554121,18.883316,11.529660,789040,2541.736441,2541.736441,613241
1,108,5,01,79.840469,19.636886,19.227926,918940,82.639152,82.639152,839466
2,103,36,25,52.515885,19.705162,11.554691,783077,259.448137,259.448137,672360
3,105,22,10,49.759906,19.058262,22.359293,725883,146.075678,146.075678,608170
4,109,1,17,88.179594,19.672451,20.205281,644610,111.310852,111.310852,565132
...,...,...,...,...,...,...,...,...,...,...
16563,101,6,21,80.184022,20.549793,18.591359,847865,143.314010,143.314010,703978
16564,113,7,21,88.179594,17.957862,18.591359,847865,143.314010,143.314010,703978
16565,115,3,21,88.179594,20.576569,18.591359,847865,143.314010,143.314010,703978
16566,108,6,21,77.788805,19.670205,18.591359,847865,143.314010,143.314010,703978


In [113]:
bqclient.load_table_from_dataframe?

Signature:
bqclient.load_table_from_dataframe(
    dataframe,
    destination,
    num_retries=6,
    job_id=None,
    job_id_prefix=None,
    location=None,
    project=None,
    job_config=None,
    parquet_compression='snappy',
    timeout=None,
)
Docstring:
Upload the contents of a table from a pandas DataFrame.

Similar to :meth:`load_table_from_uri`, this method creates, starts and
returns a :class:`~google.cloud.bigquery.job.LoadJob`.

.. note::

    Due to the way REPEATED fields are encoded in the ``parquet`` file
    format, a mismatch with the existing table schema can occur, and
    100% compatibility cannot be guaranteed for REPEATED fields.

    https://github.com/googleapis/python-bigquery/issues/17

Args:
    dataframe (pandas.DataFrame):
        A :class:`~pandas.DataFrame` containing the data to load.
    destination (google.cloud.bigquery.table.TableReference):
        The destination table to use for loading the data. If it is an
        existing table, the schema o

In [79]:
query = f"""
    select
        *
    from (
        {stats_stack}
        )
"""
run_query(query)

,seed,plan,cd,polsby_popper,total_pop
0,114,0,01,19.400764,722363
1,114,1,01,19.400764,722363
2,114,2,01,19.400764,722363
3,114,3,01,19.400764,722363
4,114,4,01,19.400764,722363
...,...,...,...,...,...
17817,100,26,38,26.350145,882273
17818,100,27,38,26.350145,882273
17819,100,28,38,26.350145,882273
17820,100,29,38,26.350145,882273


In [67]:
df

,seed,plan,hash,pop_imbalance,polsy_popper,r
0,114,5,-7915478809046984484,67.880162,19.059284,1
1,104,5,-6709255699811009922,84.423077,20.401792,1
2,103,14,-6459736489883378198,65.249087,18.834906,1
3,108,7,-6393807558829545116,77.788805,19.582134,1
4,111,25,-5541735169537300084,50.307764,17.205912,1
...,...,...,...,...,...,...
450,108,35,-4037798534409500956,55.848145,18.065215,1
451,115,43,-2188271141291405578,52.394110,17.920852,1
452,107,17,756676115651280966,57.830578,18.232746,1
453,105,18,3543382720719122944,65.375295,18.984034,1


In [64]:
tbl = f'cmat-315920.redistricting_results_cook.TX_2020_cntyvtd_cd_seed'
sels = [f'select * from {tbl}_{str(seed).rjust(4, "0")}_summary' for seed in range (100, 116) if seed != 110]
query = "\nunion all\n".join(sels)
query = f"""
    select
        A.hash,
        max(seed),
        max(plan),
        count(*) as count
    from (
        {query}
        ) as A
    group by
        A.hash
"""
df = run_query(query)
# 
df['hash'].value_counts().sort_values(ascending=False)

-9000968703311639662    1
 1686731019043959074    1
 5326950720663711766    1
 3782232777236031018    1
-2692892709489472754    1
                       ..
-8883311163469327286    1
 4073746617601787200    1
 588406201666064194     1
 2579663012286536464    1
 2357127729327828404    1
Name: hash, Length: 455, dtype: int64

In [56]:
df.dtypes

seed               int64
plan               int64
hash               int64
pop_imbalance    float64
polsy_popper     float64
r                  int64
dtype: object

In [63]:
# df['seed'].unique()



-9000968703311639662    15
-3612179245007624834     1
 8819512477695847480     1
-8088312543324601526     1
-7104331499606189002     1
                        ..
 4209038618835388202     1
 7292070618630616890     1
-6120954419377883746     1
-5055405117699558834     1
 2731995706378505462     1
Name: hash, Length: 455, dtype: int64